<a href="https://colab.research.google.com/github/sheon-j/nlp-study/blob/main/tensorflow-nlp-tutorial/week02_count_based_word_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Study: Week 2 - Count based word Representation

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 스터디

---

**Contents**
1. 다양한 단어의 표현 방법
2. Bag of Words(BoW)
3. 문서 단어 행렬(Document-Term Matrix, DTM)
4. TF-IDF(Term Frequency-Inverse Document Frequency)

## 1. 다양한 단어의 표현 방법
* 여러 문서로 이루어진 텍스트 데이터를 통계적으로 접근하는 방법
* 중요도를 표현하거나, 핵심어를 추출, 검색 엔진의 순위 결정, 문서들 간의 유사도를 구하는 등 용도로 사용 가능

### 1.1. 단어의 표현 방법
#### (1) 국소 표현 (Local Representation)
* a.k.a 이산 표현 (Discrete Representation)
* 단어 그 자체만 보고 특정 값을 맵핑하여 표현

#### (2) 분산 표현
* a.k.a 연속 표현 (Continuous Represnetation)
* 단어를 표현하고자 주변을 참고하여 표현
* 단어의 의미, 뉘앙스를 표현 (ex. 귀여운 강아지)

### 1.2. 단어 표현의 카테고리화

![word-category](https://wikidocs.net/images/page/31767/wordrepresentation.PNG)

* 국소표현
  * Bag of Words : 단어의 빈도수를 카운트하여 단어를 수치화하는 표현법
  * DTM: Bag of Words의 확장 (TDM)
  * TF-IDF: 빈도수 기반 단어 표현 + 중요도(가중치)
* 연속표현
  * Word2Vec: 예측을 기반으로 단어의 뉘앙스 표현
  * FastText: Word2Vec의 확장
  * GloVe: 예측과 카운트가 모두 사용된 표현법


## 2. Bag of Words(BoW)

* 단어의 등장 순서를 고려하지 않는 빈도수 기반의 단어 표현 방법

### 2.1. Bag of Words란?
* 등장한 횟수를 수치화하는 텍스트 표현 방법
* 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단
  * 분류 문제나 여러 문서 간의 유사도를 구하는 문제
  * '달리기', '체력', '근력' => 체육 관련 문서
  * '미분', '방정식', '부등식'=> 수학 관련 문서
* BoW를 만드는 과정
  1. 각 단어에 고유한 정수 인덱스 부여
  2. 각 인덱스의 위치에 단어 토큰 빈도를 기록한 벡터 생성

* 예제 문서1 : **정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.**

In [ ]:
!pip install konlpy

In [3]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
  # 온점 제거 및 형태소 분석
  document = document.replace('.', '')
  tokenized_document = okt.morphs(document)

  word_to_index = {}
  bow = []

  for word in tokenized_document:  
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)  
      # BoW에 전부 기본값 1을 넣는다.
      bow.insert(len(word_to_index) - 1, 1)
    else:
      # 재등장하는 단어의 인덱스
      index = word_to_index.get(word)
      # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
      bow[index] = bow[index] + 1

  return word_to_index, bow

In [4]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


### 2.2. 다양한 예제들
* 예제 문서2 : **소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.**


In [5]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


* 예제 문서3: 문서1 + 문서2

In [6]:
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


### 2.3. CountVectorizer 클래스로 BoW 만들기

* 사이킷 런의 CountVectorizer 클래스: 단어의 빈도를 Count하여 벡터화
* 영어에 한정하여 쉽게 BoW 생성
* 띄어쓰기만으로 토큰화가 수행되기 때문에 형태소 분리가 필수인 한국어엔 부적합

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()


# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


### 2.4. 불용어를 제거한 BoW
* BoW를 만들때 불용어를 제거하는 일은 자연어 처리의 정확도를 높이기 위해서 선택할 수 있는 전처리 기법

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# 사용자 직접 정의 불용어
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [18]:
# CountVectorizer에서 제공하는 자체 불용어
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


In [ ]:
# NLTK 불용어 사용
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

## 3. 문서 단어 행렬(Document-Term Matrix, DTM)

* 문서 단어 행렬(Document-Term Matrix, DTM) 표현법은 서로 다른 문서들의 BoW들을 결합한 표현 방법
*DTM 혹은 행과 열을 반대로 한 TDM이라고도 불리움
* 서로 다른 문서들을 비교 가능

### 3.1. 문서 단어 행렬(Document-Term Matrix, DTM)의 표기법

* 각기 다른 4개의 문서
  1. 먹고 싶은 사과
  2. 먹고 싶은 바나나
  3. 길고 노란 바나나 바나나
  4. 저는 과일이 좋아요
* 문서 단어 행렬

| -     | 과일이 | 길고 | 노란 | 먹고 | 바나나 | 사과 | 싶은 | 저는 | 좋아요 |
| :---- | :----- | :--- | :--- | :--- | :----- | :--- | :--- | :--- | :----- |
| 문서1 | 0      | 0    | 0    | 1    | 0      | 1    | 1    | 0    | 0      |
| 문서2 | 0      | 0    | 0    | 1    | 1      | 0    | 1    | 0    | 0      |
| 문서3 | 0      | 1    | 1    | 0    | 2      | 0    | 0    | 0    | 0      |
| 문서4 | 1      | 0    | 0    | 0    | 0      | 0    | 0    | 1    | 1      |

*  문서 단어 행렬은 문서들을 서로 비교할 수 있도록 수치화할 수 있다는 점에서 의의

### 3.2. 문서 단어 행렬(Document-Term Matrix)의 한계

#### (1) 희소 표현(Sparse representation)
* 긱 문서 벡터의 차원은 전체 단어 집합의 크기를 가짐
 * 전체 코퍼스가 방대한 데이터 - 문서 벡터의 차원은 수만 이상의 차원
* 대부분의 값이 0인 표현을 희소 벡터(sparse vector) 또는 희소 행렬(sparse matrix)라 불리움: 원-핫 벡터, DTM
* 희소 벡터는 많은 양의 저장 공간과 높은 계산 복잡도를 요구합
* 집합의 크기를 줄이는 전처리 방법 중요
  * 구두점, 빈도수가 낮은 단어, 불용어 제거, 어간추출 및 표제어 추출을 통해 단어를 정규화

#### (2) 단순 빈도 수 기반 접근
* 모든 단어에 대해서 빈도 표기를 하는 이런 방법의 한계
* 빈도수가 높더라도 자연어 처리에 있어 의미를 갖지 못하는 단어도 있음


## 4. TF-IDF(Term Frequency-Inverse Document Frequency)

* DTM 내에 있는 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF
* 기존의 DTM을 사용하는 것보다 보다 많은 정보를 고려하여 문서 비교 가능
* 많은 경우에서 DTM보다 더 좋은 성능

### 4.1. TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

* 단어의 빈도와 (문서의 빈도에 특정 식을 취하는) 역 문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
* 문서의 유사도를 구하는 작업
  * 검색 시스템에서 검색 결과의 중요도
  * 문서 내에서 특정 단어의 중요도
* TF-IDF를 만드는 과정
  1. DTM 생성
  2. TF-IDF 가중치를 부여


#### (1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
* 문서 단어 행렬 DTM

#### (2) df(t) : 특정 단어 t가 등장한 문서의 수
* 특정 단어 t가 등장한 문서의 수에만 관심
* DTM 예시에서 바나나는 문서2와 문서3에 등장. 이 경우, 바나나의 df는 2

#### (3) idf(d, t) : df(t)에 반비례하는 수

# <center>$idf(d, t) = log(\frac{n}{1+df(t)})$<center/>
* n은 총 문서의 수
* IDF를 단순 역수($\frac{n}{df(t)}$)로 사용한다면 n이 커질 수록 IDF의 값은 기하급수적으로 증가
  * log를 씌워주지 않으면, 희귀 단어들에 엄청난 가중치가 부여
  * log를 씌워 격차를 줄임
* 분모에 1을 더해 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지

* 전체문서 1,000,000개일 때, idf 수식 비교

| 단어 t  | df(t)   | n/df(d, t) | log(n/df(t)) |
| :----- | :------ | :--------- | ------------ |
| word1  | 1       | 1,000,000  | 6            |
| word2  | 100     | 10,000     | 5            |
| word3  | 1,000   | 1,000      | 4            |
| word4  | 10,000  | 100        | 2            |
| word5  | 100,000 | 10         | 1            |

* tf-idf 과정

| tf     | 과일이 | 길고 | 노란 | 먹고 | 바나나 | 사과 | 싶은 | 저는 | 좋아요 |
| :---- | :----- | :--- | :--- | :--- | :----- | :--- | :--- | :--- | :----- |
| 문서1 | 0      | 0    | 0    | 1    | 0      | 1    | 1    | 0    | 0      |
| 문서2 | 0      | 0    | 0    | 1    | 1      | 0    | 1    | 0    | 0      |
| 문서3 | 0      | 1    | 1    | 0    | 2      | 0    | 0    | 0    | 0      |
| 문서4 | 1      | 0    | 0    | 0    | 0      | 0    | 0    | 1    | 1      |

| idf  | 과일이      | 길고        | 노란        | 먹고        | 바나나      | 사과        | 싶은        | 저는        | 좋아요      |
| :--- | :---------- | :---------- | :---------- | :---------- | :---------- | :---------- | :---------- | :---------- | :---------- |
| 수식 | ln(4/(1+1)) | ln(4/(1+1)) | ln(4/(1+1)) | ln(4/(2+1)) | ln(4/(2+1)) | ln(4/(1+1)) | ln(4/(2+1)) | ln(4/(1+1)) | ln(4/(1+1)) |
| 값   | 0.693147    | 0.693147    | 0.693147    | 0.287682    | 0.287682    | 0.693147    | 0.287682    | 0.693147    | 0.287682    |

| tf-idf  | 과일이   | 길고     | 노란     | 먹고     | 바나나   | 사과     | 싶은     | 저는     | 좋아요   |
| :---- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| 문서1 | 0        | 0        | 0        | 0.287682 | 0        | 0.693147 | 0.287682 | 0        | 0        |
| 문서2 | 0        | 0        | 0        | 0.287682 | 0.287682 | 0        | 0.287682 | 0        | 0        |
| 문서3 | 0        | 0.693147 | 0.693147 | 0        | 0.575364 | 0        | 0        | 0        | 0        |
| 문서4 | 0.693147 | 0        | 0        | 0        | 0        | 0        | 0        | 0.693147 | 0.693147 |

### 4.2. 파이썬으로 TF-IDF 직접 구현하기

In [19]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [20]:
# 총 문서의 수
N = len(docs) 

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

In [22]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [23]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [24]:
result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 4.3. 사이킷런을 이용한 DTM과 TF-IDF 실습

In [25]:
# 사이킷런의 BoW 클래스 CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [26]:
# 사이킷런의 TF-IDF 클래스 TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
